In [30]:

import pickle
import torch
import sys
import torch
from torch import nn

# Setup

### Data formatting

In [31]:
import json
import os

class Item:
    def __init__(self, key, value, type:str):
        self.key = key
        self.type = type
        if type.startswith("Tensor"):
            self.value = {"size":value.size(),"values":value.flatten().tolist()}
        else:
            self.value = value

    def to_dict(self):
        return {self.key: {self.type: self.value}}


def to_file(name:str,items:list):
    path = "test-files/"+name+".json"
    values = {}
    for item in items:
        values.update(item.to_dict())
    output = {"values": values}
    
    data = json.dumps(output, indent=4)
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "w") as f:
        f.write(data)

def random_tensor(shape:list,seed:int=0):
    n = 1 
    for dim in shape:
        n*=dim

    a = 3
    c = 23
    m = 2**32
    
    result = []
    x = seed
    for _ in range(n):
        x = (a * x + c) % m
        result.append(x / m)  # Normalize the result to [0, 1]

    return torch.tensor(result).view(shape)


## Common
#### LayerNorm2d

In [32]:
from segment_anything.modeling.common import LayerNorm2d

layer_norm = LayerNorm2d(256,0.1)
items = [Item("weight", layer_norm.weight, "TensorFloat"), Item("bias", layer_norm.bias, "TensorFloat"), Item("eps", layer_norm.eps, "Float")]
to_file("layer_norm_2d",items)

# Forward
input = random_tensor([2,256,16,16])
output = layer_norm(input)
items = [Item("input", input, "TensorFloat"), Item("output", output, "TensorFloat")]
to_file("layer_norm_2d_forward",items)

#### MLPBlock

In [33]:
from segment_anything.modeling.common import MLPBlock

mlp_block = MLPBlock(256,256,nn.GELU)
mlp_block.lin1.weight.data = random_tensor([256,256],1)
mlp_block.lin2.weight.data = random_tensor([256,256],2)
mlp_block.lin1.bias.data = random_tensor([256],3)
mlp_block.lin2.bias.data = random_tensor([256],4)
items=[Item("lin1", mlp_block.lin1.weight, "TensorFloat"), Item("lin2", mlp_block.lin2.weight, "TensorFloat")]
to_file("mlp_block",items)

# Forward
input = random_tensor([256,256],5)
output = mlp_block(input)
items = [Item("input", input, "TensorFloat"), Item("output", output, "TensorFloat")]
to_file("mlp_block_forward",items)

#### Activation

In [34]:
# Gelu
gelu = nn.GELU()
input = random_tensor([256,256])
output = gelu(input)
items = [Item("input", input, "TensorFloat"), Item("output", output, "TensorFloat")]
to_file("activation_gelu",items)

# ReLU
relu = nn.ReLU()
input = random_tensor([256,256])
output = relu(input)
items = [Item("input", input, "TensorFloat"), Item("output", output, "TensorFloat")]
to_file("activation_relu",items)


# Image encoder

#### PatchEmbeded

In [35]:
from segment_anything.modeling.image_encoder import PatchEmbed

patch_embed = PatchEmbed((16,16),(16,16),(0,0),3,768)
patch_embed.proj.weight.data = random_tensor([2,256,16,16],1)
patch_embed.proj.bias.data = random_tensor([2],2)
items=[Item("weight", patch_embed.proj.weight, "TensorFloat"),Item("bias", patch_embed.proj.bias, "TensorFloat")]
to_file("patch_embed",items)

# Forward
input = random_tensor([2,256,16,16],3)
output = patch_embed(input)
items = [Item("input", input, "TensorFloat"), Item("output", output, "TensorFloat")]
to_file("patch_embed_forward",items)


#### Block

In [36]:
from segment_anything.modeling.image_encoder import  window_partition,window_unpartition

# Window partition
input = random_tensor([2,256,16,16],1)
output,size = window_partition(input,16)
items = [Item("input", input, "TensorFloat"), Item("output", output, "TensorFloat"), Item("size", size, "Size")]
to_file("window_partition",items)

# Window unpartition
input = random_tensor([2,256,16,16],2)
output = window_unpartition(input,16,(16,16),(14,14))
items = [Item("input", input, "TensorFloat"), Item("output", output, "TensorFloat")]
to_file("window_unpartition",items)

torch.Size([400, 196, 80]) torch.Size([400, 196, 80]) torch.Size([400, 196, 80])
torch.Size([400, 196, 80]) torch.Size([400, 80, 196])


In [53]:
from segment_anything.modeling.image_encoder import Block

#Block
block = Block(1280,16,4.0,True,nn.LayerNorm,nn.GELU,True,True,14,(64,64))
items=[Item("window_size", block.window_size, "Int")]
to_file("block",items)

#Forward
input = random_tensor([1,64,64,1280],1)
block.norm1.weight.data = random_tensor([1280],2)
block.norm1.bias.data = random_tensor([1280],3)
block.norm2.weight.data = random_tensor([1280],4)
block.norm2.bias.data = random_tensor([1280],5)
block.attn.qkv.weight.data = random_tensor([3840,1280],6)
block.attn.qkv.bias.data = random_tensor([3840],7)
block.attn.proj.weight.data = random_tensor([1280,1280],8)
block.attn.proj.bias.data = random_tensor([1280],9)
block.mlp.lin1.weight.data = random_tensor([5120,1280],10)
block.mlp.lin1.bias.data = random_tensor([5120],11)
block.mlp.lin2.weight.data = random_tensor([1280,5120],12)
block.mlp.lin2.bias.data = random_tensor([1280],13)
output = block(input)
items = [Item("input", input, "TensorFloat"), Item("output", output, "TensorFloat")]
to_file("block_forward",items)

torch.Size([400, 196, 80]) torch.Size([400, 196, 80]) torch.Size([400, 196, 80])
torch.Size([400, 196, 80]) torch.Size([400, 80, 196])


#### Attention

In [37]:
from segment_anything.modeling.image_encoder import get_rel_pos,add_decomposed_rel_pos

# Get rel pos
q_size = 64
k_size = 64
input = random_tensor([127,80],1)
output = get_rel_pos( q_size, k_size, input)
items = [Item("input",input,"TensorFloat"),Item("output", output, "TensorFloat")]
to_file("get_rel_pos",items)

# Add decomposed rel pos
attn = random_tensor([400,196,196],2)
q = random_tensor([400,196,80],3)
relo_pos_h = random_tensor([27,80],4)
relo_pos_w = random_tensor([27,80],5)
q_size = (14,14)
k_size = (14,14)
rel_pos = add_decomposed_rel_pos(attn,q,relo_pos_h,relo_pos_w,q_size,k_size)
items = [Item("attn", attn, "TensorFloat"), Item("q", q, "TensorFloat"), Item("q_size", q_size, "Size"), Item("k_size", k_size, "Size"), Item("output", rel_pos, "TensorFloat")]
to_file("add_decomposed_rel_pos",items)

In [38]:
from segment_anything.modeling.image_encoder import Attention

# Attention
attention = Attention(1280, 16 ,True ,True ,True, (14, 14))
items =[Item("num_heads", attention.num_heads, "Int"), Item("scale", attention.scale, "Float"),  Item("use_rel_pos", attention.use_rel_pos, "Bool")]
to_file("attention",items)

# Forward
input = random_tensor([25,14,14,1280],1)
attention.qkv.weight.data = random_tensor([3840,1280],2)
attention.qkv.bias.data = random_tensor([3840],3)
attention.proj.weight.data = random_tensor([1280,1280],4)
attention.proj.bias.data = random_tensor([1280],5)
output = attention(input)
items = [Item("input", input, "TensorFloat"), Item("output", output, "TensorFloat")]
to_file("attention_forward",items)

torch.Size([400, 196, 80]) torch.Size([400, 196, 80]) torch.Size([400, 196, 80])
torch.Size([400, 196, 80]) torch.Size([400, 80, 196])
